In [2]:
# Python Script to Extract tweets of a 
# particular Hashtag using Tweepy and Pandas

# import modules
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from collections import defaultdict
import datetime
import tweepy
import json
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')  

[nltk_data] Downloading package punkt to /Users/amalie-
[nltk_data]     maria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/amalie-
[nltk_data]     maria/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# function to display data of each tweet
def printtweetdata(n, ith_tweet):
    print()
    print(f"Tweet {n}:")
    print(f"Username:{ith_tweet[0]}")
    print(f"Description:{ith_tweet[1]}")
    print(f"Location:{ith_tweet[2]}")
    print(f"Following Count:{ith_tweet[3]}")
    print(f"Follower Count:{ith_tweet[4]}")
    print(f"Total Tweets:{ith_tweet[5]}")
    print(f"Retweet Count:{ith_tweet[6]}")
    print(f"Tweet Text:{ith_tweet[7]}")
    print(f"Hashtags Used:{ith_tweet[8]}")
    print(f"Date:{ith_tweet[9]}")

In [4]:
#importer dansk som sprog
#pip install afinn
from afinn import Afinn
afinn = Afinn(language='da')

-6.0

### Combining dataframes

In [14]:
df1=pd.read_csv('df1.csv')
df2=pd.read_csv('df2.csv')
df3=pd.read_csv('df3.csv')

obs = [df1, df2, df3]

data_set = pd.concat(obs)
data_set = data_set.reset_index(drop = True)

data_set.to_csv('twitter_data.csv', encoding = 'utf-8', index= False)

data_set

,Unnamed: 0,username,description,location,following,followers,totaltweets,retweetcount,text,hashtags,date
0,0,SupAyaLaya,NaN,NaN,1228,521,34115,1,"Beder alle indberette dette tweet, da det tyde...",[],2021-08-16 23:59:33
1,1,Mullarrd,"Nearsighted scrap pile, perpetual lefty. Searc...",Pangaea,515,536,39951,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 21:14:11
2,2,GitteKJP,"Søger viden, nyheder og indsigter til eftertanke.",København,972,947,231239,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 16:41:45
3,3,idamariehansen7,NaN,Nykøbing Sjælland,1292,346,21549,17,Få din vaccination uden tidsbestilling. Der er...,[],2021-08-16 16:07:07
4,4,ThomasHeie,Fængselsbetjent - FREM'er - Fotograf - FDFer,"København, Danmark",111,269,21567,0,Indsatsen i Afghanistan har kostet Danmark 15 ...,"['dkpol', 'COVID19dk']",2021-08-16 15:48:37
...,...,...,...,...,...,...,...,...,...,...,...
1731,26,KlausKblog,"Klaus K, journalist & sangskriver i Danser med...",København,511,2464,33254,28,Israels massevaccination: \n25% stigning i hje...,[],2021-08-11 18:48:21
1732,27,HettingClaus,"""Three things remain not long hidden: The Sun,...",Denmark,2249,2717,15616,28,Israels massevaccination: \n25% stigning i hje...,[],2021-08-11 17:59:14
1733,28,nbm_1981,IT Automation Consultant (MCAAA/MCSA/MCP). Lik...,"København, Danmark",612,624,12430,28,Israels massevaccination: \n25% stigning i hje...,"['dkpol', 'dkmedier', 'coronadk', 'covid19dk',...",2021-08-11 17:50:59
1734,29,transceiverfreq,xi=aixi'-(d+ai0+ai1)xi+rxi(fi-Φ ) | Freelance ...,"Frederiksberg, Denmark",339,1089,44893,0,@DRNyheder @tv2newsdk @moderna_tx @pfizer Jeg ...,"['dkpol', 'dkmedier', 'COVID19']",2021-08-10 19:29:12


### Cleaning the DataFrame

In [15]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
         "]+", flags=re.UNICODE)

In [16]:
#Dropping collumns
data_set1 = data_set.drop(['Unnamed: 0','username', 'description', 'followers', 'following', ], axis=1).reset_index(drop = True)

#Creating collumn for hashtags
data_set1['hashtags'] = data_set1['text'].apply(lambda x: re.findall(r'#(\w+)', x))

#Calling text column
tweets = data_set1.text.values[0]

In [17]:
import numpy as np
def clean_tweet(tweet):
    if type(tweet) == np.float64:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) 
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('[.,*?:]',' ', temp)
    temp = re.sub('\v', ' ', temp)
    temp = re.sub("[\\n]",' ', temp)
    temp = re.sub('1/n',' ', temp)
    temp = re.sub('2/n',' ', temp)
    temp = re.sub('3/n',' ', temp)
    temp = re.sub('4/n',' ', temp)
    temp = re.sub('5/n',' ', temp)
    temp = re.sub('6/n',' ', temp)
    temp = re.sub('7/n',' ', temp)
    temp = re.sub('8/n',' ', temp)
    temp = re.sub('-','', temp) 
    temp = re.sub('%','', temp) 
    temp = re.sub('["]','', temp)
    temp = re.sub("[']","", temp)
    temp = re.sub('[0-9]', ' ', temp)
    temp = re.sub('\+', ' ', temp)
   
    temp = emoji_pattern.sub(r'', temp)
    return temp

In [13]:
tweets = [None for i in range(len(data_set1))]
#ny = [None for i in range(len(data_set1))]
score = [None for i in range(len(data_set1))]
for i in range(len(data_set1)):  
    tweets[i] = clean_tweet(data_set1.text.values[i])
    score[i]= afinn.score(tweets[i])


In [113]:
data_set1['Clean Text'] = tweets
data_set1['Sentiment'] = score
final_set = data_set1.drop(['text', 'clean_tweets', 'afinn_score'], axis=1).reset_index(drop = True)
final_set

,location,totaltweets,retweetcount,hashtags,date,Clean Text,Sentiment
0,NaN,34115,1,"[coronadk, dkmedier, dkpol, covid19dk]",2021-08-16 23:59:33,beder alle indberette dette tweet da det tyde...,-1.0
1,Pangaea,39951,17,"[Bornholm, sundpol, dkpol, COVID19dk]",2021-08-16 21:14:11,få din vaccination uden tidsbestilling der er...,-2.0
2,København,231239,17,"[Bornholm, sundpol, dkpol, COVID19dk]",2021-08-16 16:41:45,få din vaccination uden tidsbestilling der er...,-2.0
3,Nykøbing Sjælland,21549,17,"[Bornholm, sundpol, dkpol, COVID19dk]",2021-08-16 16:07:07,få din vaccination uden tidsbestilling der er...,-2.0
4,"København, Danmark",21567,0,"[dkpol, COVID19dk]",2021-08-16 15:48:37,indsatsen i afghanistan har kostet danmark 15 ...,0.0
...,...,...,...,...,...,...,...
1731,København,33254,28,"[dkpol, dkmedier, coronadk, covid19dk, sundpol...",2021-08-11 18:48:21,israels massevaccination 25% stigning i hjer...,0.0
1732,Denmark,15616,28,"[dkpol, dkmedier, coronadk, covid19dk, sundpol...",2021-08-11 17:59:14,israels massevaccination 25% stigning i hjer...,0.0
1733,"København, Danmark",12430,28,"[dkpol, dkmedier, coronadk, covid19dk, sundpol...",2021-08-11 17:50:59,israels massevaccination 25% stigning i hjer...,0.0
1734,"Frederiksberg, Denmark",44893,0,"[dkpol, dkmedier, COVID19]",2021-08-10 19:29:12,jeg kan på det varmeste anbefale at læse ...,2.0


In [70]:
stop_words_list = nltk.corpus.stopwords.words("danish")

sent_sw_removed = [i for i in nltk.word_tokenize(tweets) if i not in stop_words_list]
sent_sw_removed

['Beder',
 'indberette',
 'tweet',
 ',',
 'tydeligt',
 'opfordrer',
 'overgreb',
 ',',
 'tvang',
 'vold',
 'børn',
 'unge',
 '.',
 'Bør',
 'høre',
 'hjemme',
 'sociale',
 'medier',
 '#',
 'coronadk',
 '#',
 'dkmedier',
 '#',
 'dkpol',
 '#',
 'covid19dk',
 '@',
 'SSTbrostrom',
 '@',
 'Heunicke',
 'monstre',
 'skabt',
 '?',
 'https',
 ':',
 '//t.co/Ido6EZf2EC']

In [23]:
#Taking out emojis
emoji = emoji_pattern.sub(r'', str(clean_data))

#Tokenizing the data 
sent = nltk.tokenize.word_tokenize(emoji)

#Remove danish stopWords
stopWords = set(stopwords.words('danish'))
wordsFiltered = []

for w in sent:
    if w not in stopWords:
        wordsFiltered.append(w)

In [25]:
sent

['[',
 "'",
 'b',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 'r',
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 'a',
 "'",
 ',',
 "'",
 'l',
 "'",
 ',',
 "'",
 'l',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 'i',
 "'",
 ',',
 "'n",
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'b',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 'r',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 'w',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'a',
 "'",
 ',',
 "'",
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 "'",
 ',',
 "'t",
 "'",
 ',',
 "'",
 'y',
 "'",
 ',',
 "'d",
 "'",
 ',',
 "'",
 'e',
 "'",
 ',',
 "'",
 'l',
 

In [114]:
# has only been lower-cased and tokenized (a few cells above)
d = defaultdict(int)
for i in wordsFiltered:
    for w in i:
        d[w]+=1
V = d.keys()
print("Vocabulary size:",len(V))

# inspect the 10 most frequent tokens:
{k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)[:10]}

Vocabulary size: 177
